<a href="https://colab.research.google.com/github/abbaspouraa/Automated-Cardiac-Diagnosis/blob/main/2D_Segmentation_SM_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install segmentation_models
!pip install tensorflow

from IPython.display import clear_output
clear_output()

import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

from tensorflow.keras.utils import normalize
from tensorflow.keras.metrics import MeanIoU

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # !pip install segmentation_models

# import segmentation_models as sm
# import os, glob
# import numpy as np
# import matplotlib.pyplot as plt
# import tensorflow as tf
# from tensorflow.keras.utils import to_categorical
# from google.colab import drive
# drive.mount('/content/drive')
# from keras.metrics import MeanIoU

# sm.set_framework('tf.keras')
# sm.framework()

Segmentation Models: using `keras` framework.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'tf.keras'

In [ ]:
all_img = sorted (glob.glob('/content/drive/MyDrive/Data/CardioData/Dataset/training/*/*frame*.nii.gz'))
mask_list = sorted (glob.glob('/content/drive/MyDrive/Data/CardioData/Dataset/training/*/*gt.nii.gz'))
img_list = [img for img in all_img if img not in mask_list]

In [ ]:
# import cv2
# SLICE_DECIMATE_IDENTIFIER = 3

# def saveSlice(img, fname, path, norm=False):
#         if norm:
#             img = np.uint8(img * 255)
            
#         fout = os.path.join(path, f'{fname}.png')
#         cv2.imwrite(fout, img)

# def sliceAndSaveVolumeImage(vol, fname, path, norm=False):
#     (dimx, dimy, dimz) = vol.shape
#     cnt = 0
#     cnt+= dimz
#     for i in range (dimz):
#         saveSlice(vol[:,:,i], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_z', path, norm)

# import nibabel as nib
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

# imageSliceOutput = '/content/drive/MyDrive/Data/CardioData/Preprocessed_Data/image'
# maskSliceOutput = '/content/drive/MyDrive/Data/CardioData/Preprocessed_Data/mask'

# for img in range (len(img_list)):
#     print ("Now we are preparong images and masks: ", img)
    
#     image = nib.load(img_list[img]).get_fdata()
#     image = scaler.fit_transform(image.reshape(-1, image.shape[-1])).reshape(image.shape)
#     sliceAndSaveVolumeImage(image, 'patient'+str((img)//2)+str(img%2), imageSliceOutput, norm=True)
    
    
#     mask = nib.load(mask_list[img]).get_fdata()
# #     mask = scaler.fit_transform(mask.reshape(-1, mask.shape[-1])).reshape(mask.shape)
#     mask = mask.astype(np.uint8)
#     sliceAndSaveVolumeImage(mask, 'patient'+str((img)//2)+str(img%2), maskSliceOutput, norm=False)

In [ ]:
import cv2
train_image = []
for directory_path in glob.glob('/content/drive/MyDrive/Data/CardioData/Preprocessed_Data/image'):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path,0)
        if img is None:
            print("None")
        else:
            img = cv2.resize(img,dsize=(256,256))
            train_image.append(img)
train_image = np.array(train_image)

In [ ]:
train_mask = []
for directory_path in glob.glob('/content/drive/MyDrive/Data/CardioData/Preprocessed_Data/mask'):
    for msk_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(msk_path,0)
        if mask is None:
            print("None")
        else:
            mask = cv2.resize(mask,dsize=(256,256), interpolation = cv2.INTER_NEAREST)
            train_mask.append(mask)
train_mask = np.array(train_mask)

In [ ]:
#Encode labels... but multi dim array so need to flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_mask.shape
train_masks_reshaped = train_mask.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 1, 2, 3])

In [ ]:
#train_images = np.expand_dims(train_images, axis=3)
#train_images = normalize(train_images, axis=1)

train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

#Create a subset of data for quick testing
#Picking 10% for testing and remaining for training
from sklearn.model_selection import train_test_split
X1, X_test, y1, y_test = train_test_split(train_image, train_masks_input, test_size = 0.10, random_state = 0)

#Further split training data t a smaller subset for quick testing of models
X_train, X_do_not_use, y_train, y_do_not_use = train_test_split(X1, y1, test_size = 0.5, random_state = 0)

print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background/few unlabeled 

Class values in the dataset are ...  [0 1 2 3]


In [ ]:
n_classes=4

from tensorflow.keras.utils import to_categorical
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))

test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

In [ ]:
# from tensorflow.keras.utils import normalize

# print(train_image.shape)
# train_image = np.expand_dims(train_image, axis = 3)
# print(train_image.shape)
# train_image = normalize(train_image, axis=1)
# train_mask = np.expand_dims(train_mask, axis = 3)

(1902, 256, 256)
(1902, 256, 256, 1)


In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(train_image, train_mask, test_size=100, random_state=0)

In [ ]:
# n_classes = 4

# train_mask_cat = to_categorical(y_train,num_classes=n_classes)
# y_train_cat = train_mask_cat.reshape((y_train.shape[0],y_train.shape[1],y_train.shape[2], n_classes))

# test_mask_cat = to_categorical(y_test,num_classes=n_classes)
# y_test_cat = test_mask_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

In [ ]:
# # Data augmetation
# from keras.preprocessing.image import ImageDataGenerator

# seed = 24

# img_data_gen_args = dict(rotation_range=60,
#                     #  width_shift_range=0.3,
#                     #  height_shift_range=0.3,
#                      shear_range=0.1,
#                      zoom_range=0.2,
#                      horizontal_flip=False,
#                      vertical_flip=False,
#                     fill_mode='constant')

# mask_data_gen_args = dict(rotation_range=60,
#                     #  width_shift_range=0.3,
#                     #  height_shift_range=0.3,
#                      shear_range=0.1,
#                      zoom_range=0.2,
#                      horizontal_flip=False,
#                      vertical_flip=False,
#                      fill_mode='constant',
#                      preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype)) #Binarize the output again. 

# image_data_generator = ImageDataGenerator(**img_data_gen_args)
# image_data_generator.fit(X_train, augment=True, seed=seed)

# image_generator = image_data_generator.flow(X_train, seed=seed)
# valid_img_generator = image_data_generator.flow(X_test, seed=seed, batch_size=32)

# mask_data_generator = ImageDataGenerator(**mask_data_gen_args)
# mask_data_generator.fit(y_train_cat, augment=True, seed=seed)
# mask_generator = mask_data_generator.flow(y_train_cat, seed=seed)
# valid_mask_generator = mask_data_generator.flow(y_test_cat, seed=seed, batch_size=32)

# def my_image_mask_generator(image_generator, mask_generator):
#     train_generator = zip(image_generator, mask_generator)
#     for (img, mask) in train_generator:
#         yield (img, mask)

# my_generator = my_image_mask_generator(image_generator, mask_generator)

# validation_datagen = my_image_mask_generator(valid_img_generator, valid_mask_generator)

In [ ]:
# x = image_generator.next()
# y = mask_generator.next()
# for i in range(0,1):
#     image = x[i]
#     mask = y[i]
#     plt.figure(figsize=(20, 15), dpi=80)
#     plt.subplot(1,4,1)
#     plt.imshow(image[:,:,0], cmap='gray')
#     plt.subplot(1,4,2)
#     plt.imshow(mask[:,:,1])
#     plt.subplot(1,4,3)
#     plt.imshow(mask[:,:,2])
#     plt.subplot(1,4,4)
#     plt.imshow(mask[:,:,3])
#     plt.show()

In [ ]:
activation = 'softmax'
LearningRate = 1e-4
opimizer = tf.keras.optimizers.Adam(LearningRate)
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.25, 0.25, 0.25, 0.25]))
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.25, 0.25, 0.25, 0.25]))
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (0.5*focal_loss)

# total_loss can be imported directly from library
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
from classification_models.keras import Classifiers

ResNet18, preprocess_input = Classifiers.get('resnet18')
resnet = ResNet18((170, 170, 3), include_top=False)

In [ ]:
sm.set_framework('tf.keras')
sm.framework()

'tf.keras'

In [ ]:
X_train_resnet.shape

(855, 256, 256)

In [ ]:
# Backbone Resnet

BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)

X_train_resnet = preprocess_input1(X_train)
X_test_resnet = preprocess_input1(X_test)

model_resnet = sm.Unet(BACKBONE1,
                       input_shape=(855, 256, 256),
                       encoder_weights='imagenet',
                       classes=n_classes,
                       activation='softmax')

model_resnet.compile(opimizer, total_loss, metrics = metrics)

# print(model_resnet.summary())

ValueError: ignored

In [ ]:
history_resnet=model_resnet.fit(X_train_resnet, 
                                y_train_cat,
                                batch_size=80, 
                                epochs=50,
                                verbose=1,
                                validation_data=(X_test_resnet, y_test_cat))


model_resnet.save('res34_backbone_50epochs.hdf5')

Epoch 1/50


ValueError: ignored

In [ ]:
BACKBONE2 = 'inceptionv3'
preprocess_input2 = sm.get_preprocessing(BACKBONE2)

# preprocess input
X_train_inception = preprocess_input2(X_train)
X_test_inception = preprocess_input2(X_test)

# define model
model_inception = sm.Unet(BACKBONE2, input_shape=(256, 256, 3), encoder_weights='imagenet', classes=n_classes, activation=activation)


# compile keras model with defined optimozer, loss and metrics
model_inception.compile(opimizer, total_loss, metrics)
#model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)


# print(model_inception.summary())


history2=model_inception.fit(X_train_inception, 
          y_train_cat,
          batch_size=8, 
          epochs=50,
          verbose=1,
          validation_data=(X_test_inception, y_test_cat))


model_inception.save('inceptionv3_backbone_50epochs.hdf5')

Epoch 1/50


ValueError: ignored

In [ ]:
print(X_train_inception.shape)
print(y_train_cat.shape)

(1802, 256, 256, 1)
(1802, 256, 256, 4)


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

#ModelCheckpoint callback saves a model at some interval. 
filepath="/content/drive/MyDrive/Data/CardioData/saved_models/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5" #File name includes epoch and validation accuracy.
#Use Mode = max for accuracy and min for loss. 
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
#This callback will stop the training when there is no improvement in
# the validation loss for three consecutive epochs.

#CSVLogger logs epoch, acc, loss, val_acc, val_loss
log_csv = CSVLogger('my_logs.csv', separator=',', append=False)

callbacks_list = [checkpoint, early_stop, log_csv]
# callbacks_list = [early_stop]

In [ ]:
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]

 
model = multiClass_UNet_Model(n_classes = n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss=[DICE_loss4], metrics=[DICE4])
# model.summary()

history = model.fit(my_generator,  steps_per_epoch=200, validation_steps=80, epochs=50,
                    validation_data=validation_datagen, shuffle=True,
                    callbacks = callbacks_list)
model.save('/content/drive/MyDrive/Data/CardioData/saved_models/Cardio_segmentation_model.h5')

Epoch 1/50
 63/200 [========>.....................] - ETA: 2:26 - loss: 0.9507 - DICE4: 0.0249

KeyboardInterrupt: ignored

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import random
test_img_number = random.randint(0, len(X_test))
test_img = X_test[test_img_number]
ground_truth=y_test[test_img_number]
test_img_norm=test_img[:,:,0][:,:,None]
test_img_input=np.expand_dims(test_img_norm, 0)
prediction = (model.predict(test_img_input))
predicted_img=np.argmax(prediction, axis=3)[0,:,:]


plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='jet')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(predicted_img, cmap='jet')
plt.show()